In [6]:
import numpy as np
import pandas as pd
import json
import os
import math
import datetime as dt
from numpy import newaxis
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from keras.models import Sequential, load_model
# from keras.utils import plot_model
# from IPython.display import SVG
# from keras.utils import model_to_dot

import utils
import model_funcs

%load_ext autoreload
# %aimport -utils -model_funcs
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
save_dir = 'models/lstm1'
df = pd.read_csv('data_25.csv', index_col="Date")
data = df.values

In [8]:
num_input = 30
num_out = 7
train_X, train_y, dev_X, dev_y, test_X, test_y, y = utils.prepare_data(data, num_input, num_preds=num_out)

print("train shape", train_X.shape)
print("predict shape", train_y.shape)

train shape (638, 30, 25)
predict shape (638, 7)


In [14]:
if not os.path.exists(save_dir): os.makedirs(save_dir)

input_shape = train_X.shape[1:]
lr = 0.001
decay = 1e-6
dropout_rate = 0.2
epochs = 25
batch_size = 32

model = Sequential()
model.add(
    LSTM(
        128,
        input_shape=input_shape,
        return_sequences=True,
    )
)
model.add(Dropout(dropout_rate))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(dropout_rate))
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(dropout_rate))

model.add(Dense(num_out, activation="softmax"))


opt = Adam(lr=lr, decay=decay)
model.compile(loss="mse", optimizer=opt)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 30, 128)           78848     
_________________________________________________________________
dropout_29 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 30, 128)           512       
_________________________________________________________________
lstm_23 (LSTM)               (None, 30, 128)           131584    
_________________________________________________________________
dropout_30 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 30, 128)           512       
_________________________________________________________________
lstm_24 (LSTM)               (None, 128)               131584    
__________

In [15]:
history = model_funcs.train(
    model,
    train_X,
    train_y,
    (dev_X, dev_y),
    epochs = epochs,
    batch_size = batch_size,
    save_dir = save_dir
)

[Model] Training Started
[Model] 25 epochs, 32 batch size
Instructions for updating:
Use tf.cast instead.
Train on 638 samples, validate on 80 samples
Epoch 1/25
638/638 [==============================] - 5s 9ms/step - loss: 0.1048 - val_loss: 0.0079
Epoch 2/25
638/638 [==============================] - 2s 3ms/step - loss: 0.1009 - val_loss: 0.0079
Epoch 3/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0986 - val_loss: 0.0069
Epoch 4/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0935 - val_loss: 0.0075
Epoch 5/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0906 - val_loss: 0.0059
Epoch 6/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0855 - val_loss: 0.0057
Epoch 7/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0837 - val_loss: 0.0056
Epoch 8/25
638/638 [==============================] - 2s 3ms/step - loss: 0.0815 - val_loss: 0.0059
Epoch 9/25
638/638 [=============================